In [1]:
import os
import glob
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.conf import SparkConf

conf = SparkConf()\
        .setAppName('spark-sql')\
        .set('spark.driver.extraClassPath', '/home/lab16/Final_PJT/data/mysql-connector-java-8.0.26.jar')

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [2]:
# 실시간 교통량 json 데이터 받아오기
path = '/home/lab16/traffic'
file_ = os.listdir(path)
file_json = [file for file in file_ if file.endswith('.json')]

file_json

['20211007000502traffic_A-17.json',
 '20211007000505traffic_A-08.json',
 '20211007000503traffic_A-15.json',
 '20211007000502traffic_A-05.json',
 '20211007000503traffic_A-07.json']

In [3]:
processing_list = []

# json 데이터 처리
for i in range(len(file_json)):
    with open('/home/lab16/traffic/%s' % (file_json[i]), 'r') as f:
        j = json.load(f)
        j_list = j['VolInfo']['row']

    # 하나의 지점 - 하나의 교통량 형태로 만들기
    sum = 0
    for i in range(len(j_list)):
        sum += int(j_list[i]['vol'])
        #print(j_list[i])
    sum //= 2

    d = dict()
    d['spot_num'] = j_list[0]['spot_num']
    d['ymd'] = j_list[0]['ymd']
    d['hh'] = j_list[0]['hh']
    d['vol'] = sum
    
    if j_list[0]['spot_num'] == 'A-08':
        d['spot_name'] = '종로(동묘앞역)'
    elif j_list[0]['spot_num'] == 'A-07':
        d['spot_name'] = '대학로(한국방송통신대학교)'
    elif j_list[0]['spot_num'] == 'A-15':
        d['spot_name'] = '종로(종로3가역)'
    elif j_list[0]['spot_num'] == 'A-17':
        d['spot_name'] = '세종대로(시청역2)'
    else:
        d['spot_name'] = '율곡로(안국역)'
        
    processing_list.append(d)
    
processing_list

KeyError: 'row'

In [4]:
# processing_list를 spark dataframe으로 변환
j_sdf = spark.createDataFrame(processing_list)
j_sdf = j_sdf.select(j_sdf.ymd, j_sdf.hh, j_sdf.spot_num, j_sdf.vol, j_sdf.spot_name)

j_sdf.createOrReplaceTempView('j_sdf')
j_sdf.show()

+--------+---+--------+----+
|     ymd| hh|spot_num| vol|
+--------+---+--------+----+
|20210928| 11|    A-08|1323|
|20210928| 11|    A-07| 903|
|20210928| 11|    A-15|1174|
|20210928| 11|    A-17|1911|
|20210928| 11|    A-05|1536|
+--------+---+--------+----+



In [5]:
# MySQL로부터 테이블 호출
realtime_traffic = spark.read.format('jdbc')\
        .option('driver', 'com.mysql.jdbc.Driver')\
        .option('url', 'jdbc:mysql://{}:3306/{}'.format('3.37.159.174', 'finalPJT'))\
        .option('user', 'mulcam')\
        .option('password', 'mulcam')\
        .option('dbtable', 'realtime_traffic')\
        .load()

# spark에서 SQL 쿼리를 위해 테이블 선언
realtime_traffic.createOrReplaceTempView('realtime_traffic')
realtime_traffic.show()

+--------+--------+--------+----+
|baseDate|baseTime|spot_num| vol|
+--------+--------+--------+----+
|20210928|      11|    A-17|1911|
|20210928|      11|    A-05|1536|
|20210928|      11|    A-07| 903|
|20210928|      11|    A-08|1323|
|20210928|      11|    A-15|1174|
+--------+--------+--------+----+



In [6]:
# 데이터 삽입
spark.sql("INSERT INTO realtime_traffic SELECT * FROM j_sdf")

DataFrame[]

In [7]:
realtime_traffic.show()

+--------+--------+--------+----+
|baseDate|baseTime|spot_num| vol|
+--------+--------+--------+----+
|20210928|      11|    A-17|1911|
|20210928|      11|    A-05|1536|
|20210928|      11|    A-07| 903|
|20210928|      11|    A-08|1323|
|20210928|      11|    A-15|1174|
|20210928|      11|    A-17|1911|
|20210928|      11|    A-05|1536|
|20210928|      11|    A-15|1174|
|20210928|      11|    A-07| 903|
|20210928|      11|    A-08|1323|
+--------+--------+--------+----+



In [8]:
# 폴더 내의 날씨 json 파일 삭제
[os.remove(f) for f in glob.glob("/home/lab16/traffic/*.json")]

[None, None, None, None, None]

### .py 파일로 만들기

In [1]:
import os
import glob
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.conf import SparkConf

conf = SparkConf()\
        .setAppName('spark-sql')\
        .set('spark.driver.extraClassPath', '/home/lab16/Final_PJT/data/mysql-connector-java-8.0.26.jar')

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [13]:
# import os
# import glob
# import json
# from pyspark import SparkContext
# from pyspark.sql import SparkSession, SQLContext
# from pyspark.sql.functions import *
# from pyspark.conf import SparkConf

# conf = SparkConf()\
#         .setAppName('spark-sql')\
#         .set('spark.driver.extraClassPath', '/home/lab16/Final_PJT/data/mysql-connector-java-8.0.26.jar')

# sc = SparkContext(conf=conf)
# sqlContext = SQLContext(sc)
# spark = sqlContext.sparkSession

# 실시간 교통량 json 데이터 받아오기
path = '/home/lab16/traffic'
file_ = os.listdir(path)
file_json = [file for file in file_ if file.endswith('.json')]

try:
    
    # json 데이터 처리
    processing_list = []

    for i in range(len(file_json)):
        with open('/home/lab16/traffic/%s' % (file_json[i]), 'r') as f:
            j = json.load(f)
            j_list = j['VolInfo']['row']

        # 하나의 지점 - 하나의 교통량 형태로 만들기
        sum = 0
        for i in range(len(j_list)):
            sum += int(j_list[i]['vol'])
            #print(j_list[i])
        sum //= 2

        d = dict()
        d['spot_num'] = j_list[0]['spot_num']
        d['ymd'] = j_list[0]['ymd']
        d['hh'] = j_list[0]['hh']
        d['vol'] = sum

        if j_list[0]['spot_num'] == 'A-08':
            d['spot_name'] = '종로(동묘앞역)'
        elif j_list[0]['spot_num'] == 'A-07':
            d['spot_name'] = '대학로(한국방송통신대학교)'
        elif j_list[0]['spot_num'] == 'A-15':
            d['spot_name'] = '종로(종로3가역)'
        elif j_list[0]['spot_num'] == 'A-17':
            d['spot_name'] = '세종대로(시청역2)'
        else:
            d['spot_name'] = '율곡로(안국역)'

        processing_list.append(d)


    # processing_list를 spark dataframe으로 변환
    j_sdf = spark.createDataFrame(processing_list)
    j_sdf = j_sdf.select(j_sdf.ymd, j_sdf.hh, j_sdf.spot_num, j_sdf.vol, j_sdf.spot_name)

    j_sdf.createOrReplaceTempView('j_sdf')


    # MySQL로부터 테이블 호출
    realtime_traffic = spark.read.format('jdbc')\
            .option('driver', 'com.mysql.jdbc.Driver')\
            .option('url', 'jdbc:mysql://{}:3306/{}'.format('3.37.159.174', 'finalPJT'))\
            .option('user', 'mulcam')\
            .option('password', 'mulcam')\
            .option('dbtable', 'realtime_traffic')\
            .load()

    # spark에서 SQL 쿼리를 위해 테이블 선언
    realtime_traffic.createOrReplaceTempView('realtime_traffic')


    # 데이터 삽입
    spark.sql("INSERT INTO realtime_traffic SELECT * FROM j_sdf")


    # 폴더 내의 날씨 json 파일 삭제
    [os.remove(f) for f in glob.glob("/home/lab16/traffic/*.json")]
    
except:
    # 폴더 내의 날씨 json 파일 삭제
    [os.remove(f) for f in glob.glob("/home/lab16/traffic/*.json")]

#### 잔여 데이터 DB에 넣기

In [1]:
import os
import glob
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.conf import SparkConf

conf = SparkConf()\
        .setAppName('spark-sql')\
        .set('spark.driver.extraClassPath', '/home/lab16/Final_PJT/data/mysql-connector-java-8.0.26.jar')

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession


path = '/home/lab16/Final_PJT/잔여traffic'
file_ = os.listdir(path)
file_json = [file for file in file_ if file.endswith('.json')]
file_json

['20210928142502traffic_A-17.json',
 '20210928121504traffic_A-08.json',
 '20210928121503traffic_A-07.json',
 '20210928142502traffic_A-15.json',
 '20210928131502traffic_A-05.json',
 '20210928131502traffic_A-17.json',
 '20210928131503traffic_A-07.json',
 '20210928142503traffic_A-07.json',
 '20210928121503traffic_A-15.json',
 '20210928142502traffic_A-05.json',
 '20210928131502traffic_A-15.json',
 '20210928152501traffic_A-17.json',
 '20210928142503traffic_A-08.json',
 '20210928121502traffic_A-17.json',
 '20210928121502traffic_A-05.json',
 '20210928152501traffic_A-05.json',
 '20210928131503traffic_A-08.json']

In [2]:
len(file_json)

17

In [3]:
# json 데이터 처리
processing_list = []

for i in range(len(file_json)):
    with open('/home/lab16/Final_PJT/잔여traffic/%s' % (file_json[i]), 'r') as f:
        j = json.load(f)
        j_list = j['VolInfo']['row']

    # 하나의 지점 - 하나의 교통량 형태로 만들기
    sum = 0
    for i in range(len(j_list)):
        sum += int(j_list[i]['vol'])
        #print(j_list[i])
    sum //= 2

    d = dict()
    d['spot_num'] = j_list[0]['spot_num']
    d['ymd'] = j_list[0]['ymd']
    d['hh'] = j_list[0]['hh']
    d['vol'] = sum
    
    if j_list[0]['spot_num'] == 'A-08':
        d['spot_name'] = '종로(동묘앞역)'
    elif j_list[0]['spot_num'] == 'A-07':
        d['spot_name'] = '대학로(한국방송통신대학교)'
    elif j_list[0]['spot_num'] == 'A-15':
        d['spot_name'] = '종로(종로3가역)'
    elif j_list[0]['spot_num'] == 'A-17':
        d['spot_name'] = '세종대로(시청역2)'
    else:
        d['spot_name'] = '율곡로(안국역)'
        
    processing_list.append(d)

processing_list

[{'spot_num': 'A-17',
  'ymd': '20210928',
  'hh': '13',
  'vol': 1678,
  'spot_name': '세종대로(시청역2)'},
 {'spot_num': 'A-08',
  'ymd': '20210928',
  'hh': '11',
  'vol': 1323,
  'spot_name': '종로(동묘앞역)'},
 {'spot_num': 'A-07',
  'ymd': '20210928',
  'hh': '11',
  'vol': 903,
  'spot_name': '대학로(한국방송통신대학교)'},
 {'spot_num': 'A-15',
  'ymd': '20210928',
  'hh': '13',
  'vol': 1204,
  'spot_name': '종로(종로3가역)'},
 {'spot_num': 'A-05',
  'ymd': '20210928',
  'hh': '12',
  'vol': 1603,
  'spot_name': '율곡로(안국역)'},
 {'spot_num': 'A-17',
  'ymd': '20210928',
  'hh': '12',
  'vol': 1699,
  'spot_name': '세종대로(시청역2)'},
 {'spot_num': 'A-07',
  'ymd': '20210928',
  'hh': '12',
  'vol': 958,
  'spot_name': '대학로(한국방송통신대학교)'},
 {'spot_num': 'A-07',
  'ymd': '20210928',
  'hh': '13',
  'vol': 996,
  'spot_name': '대학로(한국방송통신대학교)'},
 {'spot_num': 'A-15',
  'ymd': '20210928',
  'hh': '11',
  'vol': 1174,
  'spot_name': '종로(종로3가역)'},
 {'spot_num': 'A-05',
  'ymd': '20210928',
  'hh': '13',
  'vol': 1609,
  'spot

In [4]:
# processing_list를 spark dataframe으로 변환
j_sdf = spark.createDataFrame(processing_list)
j_sdf = j_sdf.select(j_sdf.ymd, j_sdf.hh, j_sdf.spot_num, j_sdf.vol, j_sdf.spot_name)

j_sdf.createOrReplaceTempView('j_sdf')


# MySQL로부터 테이블 호출
realtime_traffic = spark.read.format('jdbc')\
        .option('driver', 'com.mysql.jdbc.Driver')\
        .option('url', 'jdbc:mysql://{}:3306/{}'.format('3.37.159.174', 'finalPJT'))\
        .option('user', 'mulcam')\
        .option('password', 'mulcam')\
        .option('dbtable', 'realtime_traffic')\
        .load()

# spark에서 SQL 쿼리를 위해 테이블 선언
realtime_traffic.createOrReplaceTempView('realtime_traffic')


# 데이터 삽입
spark.sql("INSERT INTO realtime_traffic SELECT * FROM j_sdf")

DataFrame[]

#### 10월 데이터 가져오기

In [9]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import os
import json
import glob
import pandas as pd
from datetime import datetime, timedelta
import xmltodict

list_2=['A-05','A-17','A-15','A-07','A-08']

s = requests.Session()

retries = Retry(total=5,
               backoff_factor=4, # 2, 4, 8, 16, 32
               status_forcelist=[500, 502, 503, 504])

headers= {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:87.0) Gecko/20100101 Firefox/87.0'
}

In [10]:
def crawl(url, directory, name, date):
        
    res = s.get(url, headers=headers)
#    print(res.status_code)

    if res.status_code == 200:
#        print(res.content)
        result = json.loads(json.dumps(xmltodict.parse(res.content.decode('utf-8')), ensure_ascii=False))
        with open(directory + date.strftime("%Y%m%d%H%M%S") + name + ".json", "w", encoding='utf-8') as json_file:
#            print(result)
            json.dump(result, json_file, ensure_ascii=False)
        return result
    else:
        return res.status_code

In [11]:
# 비어있는 dataframe 생성
total_df = pd.DataFrame(columns=['ymd','hh','spot_num','vol','spot_name'])
total_df

,ymd,hh,spot_num,vol,spot_name


In [5]:
keys = '44636d576d63686f36344e74746170'

try:
    #for time in range(10):
        date = datetime(2021,10,8,9)   # 시작날짜, 시간
        #print(date)

        ################ 각 날짜, 시간에 해당하는 데이터 크롤링 ################
        for x in list_2:

            url = 'http://openapi.seoul.go.kr:8088/'
            queryParams = keys+'/xml'+'/VolInfo'+'/1'+'/100/'+x+'/'+date.strftime("%Y%m%d")+'/'+date.strftime("%H")

            url+=queryParams
            response= requests.get(url)

            crawl( url, '/home/lab16/traffic_9/', 'traffic_'+ x, date)

except:
    pass

In [6]:
 ################# 크롤링 이후 #################

# json 데이터 받아오기
path = '/home/lab16/traffic_9'
file_ = os.listdir(path)
file_json = [file for file in file_ if file.endswith('.json')]

file_json = sorted(file_json)
file_json

['20211008000000traffic_A-05.json',
 '20211008000000traffic_A-07.json',
 '20211008000000traffic_A-08.json',
 '20211008000000traffic_A-15.json',
 '20211008000000traffic_A-17.json',
 '20211008010000traffic_A-05.json',
 '20211008010000traffic_A-07.json',
 '20211008010000traffic_A-08.json',
 '20211008010000traffic_A-15.json',
 '20211008010000traffic_A-17.json',
 '20211008020000traffic_A-05.json',
 '20211008020000traffic_A-07.json',
 '20211008020000traffic_A-08.json',
 '20211008020000traffic_A-15.json',
 '20211008020000traffic_A-17.json',
 '20211008030000traffic_A-05.json',
 '20211008030000traffic_A-07.json',
 '20211008030000traffic_A-08.json',
 '20211008030000traffic_A-15.json',
 '20211008030000traffic_A-17.json',
 '20211008040000traffic_A-05.json',
 '20211008040000traffic_A-07.json',
 '20211008040000traffic_A-08.json',
 '20211008040000traffic_A-15.json',
 '20211008040000traffic_A-17.json',
 '20211008050000traffic_A-05.json',
 '20211008050000traffic_A-07.json',
 '20211008050000traffic_A-08

In [7]:
# 상위 5개씩 데이터 처리
idx = 0
while idx < len(file_json):
    try:
        processing_list = []

        for i in range(idx, idx+5):
            with open('/home/lab16/traffic_9/%s' % (file_json[i]), 'r') as f:
                j = json.load(f)
                j_list = j['VolInfo']['row']

            # 하나의 지점 - 하나의 교통량 형태로 만들기
            sum = 0
            for i in range(len(j_list)):
                sum += int(j_list[i]['vol'])
                #print(j_list[i])
            sum //= 2

            d = dict()
            d['spot_num'] = j_list[0]['spot_num']
            d['ymd'] = j_list[0]['ymd']
            d['hh'] = j_list[0]['hh']
            d['vol'] = sum

            if j_list[0]['spot_num'] == 'A-08':
                d['spot_name'] = '종로(동묘앞역)'
            elif j_list[0]['spot_num'] == 'A-07':
                d['spot_name'] = '대학로(한국방송통신대학교)'
            elif j_list[0]['spot_num'] == 'A-15':
                d['spot_name'] = '종로(종로3가역)'
            elif j_list[0]['spot_num'] == 'A-17':
                d['spot_name'] = '세종대로(시청역2)'
            else:
                d['spot_name'] = '율곡로(안국역)'

            processing_list.append(d)

        
        # processing_list를 spark dataframe으로 변환
        j_sdf = spark.createDataFrame(processing_list)
        j_sdf = j_sdf.select(j_sdf.ymd, j_sdf.hh, j_sdf.spot_num, j_sdf.vol, j_sdf.spot_name)

        j_sdf.createOrReplaceTempView('j_sdf')

        # MySQL로부터 테이블 호출
        realtime_traffic = spark.read.format('jdbc')\
                .option('driver', 'com.mysql.jdbc.Driver')\
                .option('url', 'jdbc:mysql://{}:3306/{}'.format('3.37.159.174', 'finalPJT'))\
                .option('user', 'mulcam')\
                .option('password', 'mulcam')\
                .option('dbtable', 'realtime_traffic')\
                .load()

        # spark에서 SQL 쿼리를 위해 테이블 선언
        realtime_traffic.createOrReplaceTempView('realtime_traffic')


        # 데이터 삽입
        spark.sql("INSERT INTO realtime_traffic SELECT * FROM j_sdf")

        idx += 5

    except:
        idx += 5

In [8]:
[os.remove(f) for f in glob.glob("/home/lab16/traffic_9/*.json")]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]